In [1]:
 !unzip iccad1_modified.zip

Streaming output truncated to the last 5000 lines.
  inflating: iccad1_modified/validation/Not_Hotspot/NNHS519.png1.png  
  inflating: __MACOSX/iccad1_modified/validation/Not_Hotspot/._NNHS519.png1.png  
  inflating: iccad1_modified/validation/Not_Hotspot/NHS60.png4.png  
  inflating: __MACOSX/iccad1_modified/validation/Not_Hotspot/._NHS60.png4.png  
  inflating: iccad1_modified/validation/Not_Hotspot/NNHS237.png4.png  
  inflating: __MACOSX/iccad1_modified/validation/Not_Hotspot/._NNHS237.png4.png  
  inflating: iccad1_modified/validation/Not_Hotspot/NHS244.png  
  inflating: __MACOSX/iccad1_modified/validation/Not_Hotspot/._NHS244.png  
  inflating: iccad1_modified/validation/Not_Hotspot/NHS49.png3.png  
  inflating: __MACOSX/iccad1_modified/validation/Not_Hotspot/._NHS49.png3.png  
  inflating: iccad1_modified/validation/Not_Hotspot/NNHS452.png2.png  
  inflating: __MACOSX/iccad1_modified/validation/Not_Hotspot/._NNHS452.png2.png  
  inflating: iccad1_modified/validation/Not_Hotspot

In [2]:
import tensorflow as tf
import os
import numpy as np
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)
import warnings
warnings.filterwarnings('ignore')

In [3]:
folder = 'iccad1_modified'

In [4]:
base_dir = os.path.join(os.path.dirname(folder), 'iccad1_modified')
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
# directory with our training hotspot pictures
train_hotspot_dir = os.path.join(train_dir, 'Hotspot')  

# directory with our training non-hotspot pictures
train_not_hotspot_dir = os.path.join(train_dir, 'Not_Hotspot')  

# directory with our validation hotspot pictures
validation_hotspot_dir = os.path.join(validation_dir, 'Hotspot')

 # directory with our non-hotspot pictures
validation_not_hotspot_dir = os.path.join(validation_dir, 'Not_Hotspot')
num_hs_tr = len(os.listdir(train_hotspot_dir))
num_nhs_tr = len(os.listdir(train_not_hotspot_dir))

num_hs_val = len(os.listdir(validation_hotspot_dir))
num_nhs_val = len(os.listdir(validation_not_hotspot_dir))

total_train = num_hs_tr + num_nhs_tr
total_val = num_hs_val + num_nhs_val

print('The dataset contains:')
print('\u2022 {:,} training images'.format(total_train))
print('\u2022 {:,} validation images'.format(total_val))

print('\nThe training set contains:')
print('\u2022 {:,} images with hotspot'.format(num_hs_tr))
print('\u2022 {:,} images without hotspot'.format(num_nhs_tr))

print('\nThe validation set contains:')
print('\u2022 {:,} images with hotspot'.format(num_hs_val))
print('\u2022 {:,} images without hotspot'.format(num_nhs_val))

The dataset contains:
• 439 training images
• 4,907 validation images

The training set contains:
• 99 images with hotspot
• 340 images without hotspot

The validation set contains:
• 227 images with hotspot
• 4,680 images without hotspot


In [5]:
!pip install timm # torch image models library

     |████████████████████████████████| 348kB 8.8MB/s 


In [6]:
import torch
import timm
import tqdm
import torch.nn as nn
from PIL import Image
import matplotlib.pyplot as plt
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [7]:
pretrained_model = "vit_base_patch16_224"
img_size = 224
epochs = 15
batch_size = 100
lr = 1e-3

In [8]:
class_dict = {0: "HS", 1: "NHS"}

In [9]:
transform = transforms.Compose([transforms.Resize((img_size, img_size)), 
                                transforms.ToTensor(), 
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [10]:
"""
BATCH_SIZE = 64
IMG_SHAPE  = 224

image_gen = ImageDataGenerator(rescale=1./255,
                              horizontal_flip = True )

train_data = image_gen.flow_from_directory(directory=train_dir,
                                          batch_size=BATCH_SIZE,
                                          shuffle=True,
                                          target_size=(IMG_SHAPE,IMG_SHAPE),
                                          class_mode='binary')

plt.imshow(train_data[0][0][0])
plt.show()
"""

"\nBATCH_SIZE = 64\nIMG_SHAPE  = 224\n\nimage_gen = ImageDataGenerator(rescale=1./255,\n                              horizontal_flip = True )\n\ntrain_data = image_gen.flow_from_directory(directory=train_dir,\n                                          batch_size=BATCH_SIZE,\n                                          shuffle=True,\n                                          target_size=(IMG_SHAPE,IMG_SHAPE),\n                                          class_mode='binary')\n\nplt.imshow(train_data[0][0][0])\nplt.show()\n"

In [11]:
"""
image_gen_val = ImageDataGenerator(rescale=1./255)

test_data = image_gen_val.flow_from_directory(directory=validation_dir,
                                                 batch_size=BATCH_SIZE,
                                                 target_size=(IMG_SHAPE, IMG_SHAPE),
                                                 class_mode='binary')
plt.imshow(test_data[0][0][0])
plt.show()
"""

"\nimage_gen_val = ImageDataGenerator(rescale=1./255)\n\ntest_data = image_gen_val.flow_from_directory(directory=validation_dir,\n                                                 batch_size=BATCH_SIZE,\n                                                 target_size=(IMG_SHAPE, IMG_SHAPE),\n                                                 class_mode='binary')\nplt.imshow(test_data[0][0][0])\nplt.show()\n"

In [12]:

train_data = datasets.ImageFolder(root="iccad1_modified/train", transform=transform)
test_data = datasets.ImageFolder(root="iccad1_modified/validation", transform=transform)

In [13]:
len(train_data), len(test_data)

(438, 4905)

In [14]:
train_batches = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_batches = DataLoader(test_data, batch_size=batch_size, shuffle=False)

In [15]:
len(train_batches), len(test_batches)

(5, 50)

In [16]:
net = timm.create_model(pretrained_model, pretrained=True)

for param in net.parameters():
    param.requires_grad = False

net.head = nn.Linear(net.head.in_features, 3)
#net.to(device)

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-vitjx/jx_vit_base_p16_224-80ecf9dd.pth" to /root/.cache/torch/hub/checkpoints/jx_vit_base_p16_224-80ecf9dd.pth


In [17]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [18]:
input = torch.randn(100, 3, 224, 224) #.to(device)
output = net(input)
output.shape

torch.Size([100, 3])

In [19]:
num_parameters = sum(p.numel() for p in net.parameters() if p.requires_grad)
num_parameters

2307

In [20]:
opt = torch.optim.Adam(net.parameters(), lr)
loss_fn = nn.CrossEntropyLoss()

In [21]:
def get_accuracy(preds, y):
    preds = preds.argmax(dim=1, keepdim=True)
    correct = preds.squeeze(1).eq(y)
    acc = correct.sum() / torch.FloatTensor([y.shape[0]]) #.to(device)

    return acc.item()

In [22]:
def loop(net, batches, train):
    batch_losses = []
    batch_accs = []

    if train:
        print("Train Loop:")
        print("")
        net.train()

        for X, y in tqdm.tqdm(batches, total=len(batches)):
            X = X #.to(device)
            y = y #.to(device)

            preds = net(X)
            loss = loss_fn(preds, y)
            acc = get_accuracy(preds, y)

            opt.zero_grad()
            loss.backward()
            opt.step()

            batch_losses.append(loss.item())
            batch_accs.append(acc)

    else:
        print("Validation Loop:")
        print("")
        net.eval()

        with torch.no_grad():
            for X, y in tqdm.tqdm(batches, total=len(batches)):
                X = X #.to(device)
                y = y #.to(device)

                preds = net(X)
                loss = loss_fn(preds, y)
                acc = get_accuracy(preds, y)

                batch_losses.append(loss.item())
                batch_accs.append(acc) 

    print("")
    print("")
    
    return sum(batch_losses) / len(batch_losses), sum(batch_accs) / len(batch_accs)

In [23]:
def predict(net, img, transform, class_dict):
    img = Image.open(img).convert("RGB")
    img = transform(img) #.to(device)
    net.eval()

    with torch.no_grad():
        pred = net(img.unsqueeze(0))
    
    pred = pred.argmax(dim=1)
    print(class_dict[pred.item()])

In [ ]:
for epoch in range(epochs):
    train_loss, train_acc = loop(net, train_batches, True)
    val_loss, val_acc = loop(net, test_batches, False)
    
    print(f"epoch: {epoch} | train_loss: {train_loss:.4f} | train_acc: {train_acc:.4f} | val_loss: {val_loss:.4f} | val_acc: {val_acc:.4f}")
    print("")

  0%|          | 0/5 [00:00<?, ?it/s]

Train Loop:



  0%|          | 0/50 [00:00<?, ?it/s]



Validation Loop:



  0%|          | 0/5 [00:00<?, ?it/s]

...

  0%|          | 0/5 [00:00<?, ?it/s]



epoch: 7 | train_loss: 0.1286 | train_acc: 0.9522 | val_loss: 0.3063 | val_acc: 0.8846

Train Loop:



  0%|          | 0/50 [00:00<?, ?it/s]



Validation Loop:



 42%|████▏     | 21/50 [24:12<33:24, 69.11s/it]